In [13]:
## Ogni .csv è strutturato in questo modo:
    # n x (12 + 4)
    # [f1, f2, ..., f12, l0, l1, l2, l3]
# dove f_i sono le features, mentre li sono le labels; n è il numero di righe, cioè di samples.
    # l0 = BASO LEFT 2 LIVELLI
    # l1 = BASO LEFT 4 LIVELLI
    # l2 = BASO RIGHT 2 LIVELLI
    # l3 = BASO RIGHT 4 LIVELLI

import csv
import statistics
from statistics import mode 
import pandas as pd
import os
import datetime
import time    # Per capire quanto tempo ci mette 
import progressbar

sliding = True
sliding_pace = 1
predict_central_sample = True

discard_transitions = False # Devo eliminare le finestre che contengono le transizioni
discarded_windows = 0
th = 1

window_size = 20
features_number = 12 # Tutto tranne il basografico
label = 0    # Label da selezionare -> Se il .csv ne contiene una sola, label = 0
row_count = 486600

start = 20    # Subject from which the count starts
end = 25    # Subject at which the count ends 


in_path = "../subjects/min-max/"
out_path = "../subjects/min-max/windows_" + str(window_size) +"/tr-" + str(discard_transitions) + "_sliding_" + str(sliding_pace) + "_c-" + str(predict_central_sample) + "/"
out_path_discarded = out_path + "discarded/"
prefix = 's'
suffix = '_norm'
out_suffix = '_norm_windows_'

#create folders if not existing...
if not os.path.exists(out_path):
    os.makedirs(out_path)
if not os.path.exists(out_path_discarded):
    os.makedirs(out_path_discarded)

for subj in range(start, end + 1):
    start_time = time.time()
    #count rows in file
    with open(in_path + prefix + str(subj) + suffix + '.csv') as csv_file:
        csv_reader = csv.reader(csv_file, delimiter=',')
        row_count = sum(1 for row in csv_reader)
    
    csv_in = []
    with open(in_path + prefix + str(subj) + suffix + '.csv') as csv_file:
        csv_reader = csv.reader(csv_file, delimiter=',')
        for line in csv_reader: 
            csv_in.append(line)
    
    with open(out_path + prefix + str(subj) + out_suffix + str(window_size) + '.csv', mode='w') as csv_out_file:
        with open(out_path_discarded + prefix + str(subj) + out_suffix + str(window_size) + '.txt', mode='w') as discarded_file:
            csv_writer = csv.writer(csv_out_file, delimiter=',')

            print("Subject " + str(subj) + " rows: " + str(row_count))

            discarded_windows = 0

            if sliding:
                pace = sliding_pace    
                limit = row_count-window_size    # Questo serve per non superare i limiti del for 
            else:
                pace = window_size
                limit = row_count
                
            for k in progressbar.progressbar(range(0,limit,pace)):    # Qui tira fuori la progressbar, per renderti conto di come cicla e di quanto tempo ci mette
                time.sleep(0.0001)    # Troppo grande rallenta il for!
                window_rows = []
                for i in range(k, k + window_size):
                    window_rows.append(csv_in[i])
                window_rows = pd.DataFrame(window_rows).astype(float)
                bsx = window_rows[features_number + label]    
                if predict_central_sample:
                    m = bsx[int(len(bsx)/2)]
                else:
                    m = bsx.mode()[0]
                  
                occ = len(window_rows[window_rows[features_number + label] == float(m)])    # Occorrenza della label nella finestra
               
                if not discard_transitions or occ/window_size >= th:       # Qua dobbiamo capire come scartarle le finestre (cioè che threshold usare!)
                    single_row_window = window_rows.iloc[:, :-4].values.reshape(1,-4).flatten().tolist()
                    single_row_window.append(float(m))
                    csv_writer.writerow(single_row_window)
                else:
                    discarded_windows += 1
                    discarded_file.write("Window starting at row " + str(k) + "\n")
                    discarded_file.write(str(window_rows) + "\n\n\n")
                    print("Discarded windows: " + str(discarded_windows))
            end_time = time.time()        
            time_tot = int(end_time - start_time)
            print('Subject ' + str(subj) + ' windowed. Time elapsed: ' + str(datetime.timedelta(seconds=time_tot)) + '\n')    
                    

  0% (29 of 408061) |                    | Elapsed Time: 0:00:00 ETA:   0:23:59

Subject 20 rows: 408081


  6% (27551 of 408061) |#                | Elapsed Time: 0:01:23 ETA:   0:19:25

KeyboardInterrupt: 

In [10]:
import time
import progressbar

for i in progressbar.progressbar(range(100)):
    time.sleep(0.01)
    
range(100)

100% (100 of 100) |######################| Elapsed Time: 0:00:01 Time:  0:00:01


range(0, 100)